# Exemplo 1: Acessando a API de localidades do IBGE

## Primeiros passos
1. Importar bibliotecas e inicializar variáveis
1. Definir como os dataframes serão exibidos

In [1]:
# importando a biblioteca:
import inteligencia_popular as ip

# vamos importar o Pandas para poder alterar as opções de exibição das tabelas
import pandas as pd

# o objeto 'ibge' nos dará acesso a todas as APIs do IBGE e também aos identificadores
ibge = ip.fontes.ibge

# o objeto 'localidades' da fonte 'ibge' fornece métodos para acessar a API de localidades
localidades = ibge.localidades

# identificadores são constantes utilizadas em diversos endpoints das APIs do IBGE
ids = ibge.identificadores

In [2]:
# definindo exibição das tabelas para não truncar o texto
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:20,.2f}'.format)
# altere o valor abaixo para mudar o tamanho máximo (em caracteres) da tabela
pd.set_option('display.width', 100)
# altere o valor abaixo para mudar o tamanho máximo (em caracteres) de cada coluna
pd.set_option('display.max_colwidth', 80)

## Divisões usadas pelo IBGE

Na [página da API de localidades do IBGE](https://servicodados.ibge.gov.br/api/docs/localidades) podemos ler o seguinte:

> API referente aos países e às divisões político-administrativas do Brasil bem como meso e microrregiões, institucionalizadas pela aprovação da presidência do IBGE da resolução PR nº 51/1989 e pela publicação [Divisão regional do Brasil em mesorregiões e microrregiões geográficas](https://biblioteca.ibge.gov.br/visualizacao/livros/liv2269_1.pdf).

Vamos ver o que nos diz a API.

In [3]:
# exibir as consultas disponíveis
print('\n'.join(f'- {c}' for c in localidades.consultas))

- distritos
- distritosPorIdentificador
- distritosPorUF
- distritosPorMesorregiao
- distritosPorMicrorregiao
- distritosPorMunicipio
- distritosPorRegiaoImediata
- distritosPorRegiaoIntermediaria
- distritosPorRegiao
- mesorregioes
- mesorregioesPorIdentificador
- mesorregioesPorUF
- mesorregioesPorRegiao
- microrregioes
- microrregioesPorIdentificador
- microrregioesPorUF
- microrregioesPorMesorregiao
- microrregioesPorRegiao
- municipios
- municipiosPorIdentificador
- municipiosPorUF
- municipiosPorMesorregiao
- municipiosPorMicrorregiao
- municipiosPorRegiaoImediata
- municipiosPorRegiaoIntermediaria
- municipiosPorRegiao
- paises
- paisesPorIdentificador
- regioes
- regioesPorIdentificador
- regioesImediatas
- regioesImediatasPorIdentificador
- regioesImediatasPorUF
- regioesImediatasPorRegiao
- regioesIntegradasDeDesenvolvimento
- regioesIntegradasDeDesenvolvimentoPorIdentificador
- regioesIntermediarias
- regioesIntermediariasPorIdentificador
- regioesIntermediariasPorUF
- regio

In [4]:
# quais informações o método 'regioes' retorna?
regioes = localidades.regioes()
print(regioes)

https://servicodados.ibge.gov.br/api/v1/localidades/regioes?view=nivelado&
   regiao-id regiao-sigla   regiao-nome
0          1            N         Norte
1          2           NE      Nordeste
2          3           SE       Sudeste
3          4            S           Sul
4          5           CO  Centro-Oeste


In [5]:
# que tal criar algumas constantes para futuras referências?
for i, reg in regioes.iterrows():
    setattr(regioes, reg['regiao-sigla'], reg['regiao-id'])
    setattr(regioes, reg['regiao-nome'].upper().replace('-', '_'), reg['regiao-id'])

print(f'NE: {regioes.NE}')
print(f'SUL: {regioes.SUL}')
print(f'CENTRO_OESTE: {regioes.CENTRO_OESTE}')

NE: 2
SUL: 4
CENTRO_OESTE: 5


In [6]:
# qual é a diferença entre mesorregião e região intermediária?
# vamos buscar os dois conjuntos apenas para o Nordeste.
# repare que o id da região é referido como 'macrorregiao' pela API
mesoregioes = localidades.mesorregioesPorRegiao(
    macrorregiao = regioes.NE)
regioes_intermediarias = localidades.regioesIntermediariasPorRegiao(
    macrorregiao = regioes.NE)

# agora vamos exibir os 5 primeiros registros de cada tabela
print('-' * 100)
print('MESORREGIÕES')
print(mesoregioes.head())
print('-' * 100)
print('REGIÕES INTERMEDIÁRIAS')
print(regioes_intermediarias.head())

https://servicodados.ibge.gov.br/api/v1/localidades/regioes/2/mesorregioes?view=nivelado&
https://servicodados.ibge.gov.br/api/v1/localidades/regioes/2/regioes-intermediarias?view=nivelado&
----------------------------------------------------------------------------------------------------
MESORREGIÕES
   mesorregiao-id   mesorregiao-nome  UF-id UF-sigla   UF-nome  regiao-id regiao-sigla regiao-nome
0            2101   Norte Maranhense     21       MA  Maranhão          2           NE    Nordeste
1            2102   Oeste Maranhense     21       MA  Maranhão          2           NE    Nordeste
2            2103  Centro Maranhense     21       MA  Maranhão          2           NE    Nordeste
3            2104   Leste Maranhense     21       MA  Maranhão          2           NE    Nordeste
4            2105     Sul Maranhense     21       MA  Maranhão          2           NE    Nordeste
----------------------------------------------------------------------------------------------------
R

In [7]:
"""
Podemos ver que tanto as mesorregiões quanto as regiões intermediárias
estão contidas em apenas um estado cada, ou seja, a única classificação
acima de UF é região (ou macrorregião).
"""
# primeiro, vamos consultar os estados e armazená-los em constantes
estados = localidades.estados()
for i, est in estados.iterrows():
    setattr(estados, est['UF-sigla'], est['UF-id'])

# agora, vamos analisar as duas classificações aplicadas à Bahia
mesoregioes = localidades.mesorregioesPorUF(
    uf = estados.BA)
regioes_intermediarias = localidades.regioesIntermediariasPorUF(
    uf = estados.BA)

# desta vez, vamos exibir todos os resultados
print('-' * 100)
print('MESORREGIÕES')
print(mesoregioes)
print('-' * 100)
print('REGIÕES INTERMEDIÁRIAS')
print(regioes_intermediarias)



https://servicodados.ibge.gov.br/api/v1/localidades/estados?view=nivelado&
https://servicodados.ibge.gov.br/api/v1/localidades/estados/29/mesorregioes?view=nivelado&
https://servicodados.ibge.gov.br/api/v1/localidades/estados/29/regioes-intermediarias?view=nivelado&
----------------------------------------------------------------------------------------------------
MESORREGIÕES
   mesorregiao-id               mesorregiao-nome  UF-id UF-sigla UF-nome  regiao-id regiao-sigla  \
0            2901           Extremo Oeste Baiano     29       BA   Bahia          2           NE   
1            2902  Vale São-Franciscano da Bahia     29       BA   Bahia          2           NE   
2            2903            Centro Norte Baiano     29       BA   Bahia          2           NE   
3            2904                Nordeste Baiano     29       BA   Bahia          2           NE   
4            2905      Metropolitana de Salvador     29       BA   Bahia          2           NE   
5            2906  

In [8]:
"""
Como podemos observar, a divisão em mesorregiões considera aspetos geográficos
(sul, centro norte, vale, região metropolitana) enquanto as regiões intermediárias
se baseiam em cidades, possivelmente.

Uma boa idéia é analisar a classificação de uma cidade conhecida e entender onde
ela se encaixa. Para esse teste, vou usar Seabra, na Bahia.
"""
municipios_bahia = localidades.municipiosPorUF(uf = estados.BA)
seabra = municipios_bahia[
    municipios_bahia['municipio-nome'] == 'Seabra'
]
print(seabra.stack())

https://servicodados.ibge.gov.br/api/v1/localidades/estados/29/municipios?view=nivelado&
367  municipio-id                           2929909
     municipio-nome                          Seabra
     microrregiao-id                          29023
     microrregiao-nome                       Seabra
     mesorregiao-id                            2906
     mesorregiao-nome             Centro Sul Baiano
     regiao-imediata-id                      290034
     regiao-imediata-nome                    Seabra
     regiao-intermediaria-id                   2910
     regiao-intermediaria-nome     Feira de Santana
     UF-id                                       29
     UF-sigla                                    BA
     UF-nome                                  Bahia
     regiao-id                                    2
     regiao-sigla                                NE
     regiao-nome                           Nordeste
dtype: object


In [9]:
# Agora vamos ver os municípios que pertencem a essas mesmas divisoes
mesorregiao = seabra['mesorregiao-id'].values[0]
regiao_intermediaria = seabra['regiao-intermediaria-id'].values[0]

municipios_mesorregiao = localidades.municipiosPorMesorregiao(mesorregiao = mesorregiao)
municipios_regiao_intermediaria = localidades.municipiosPorRegiaoIntermediaria(regiao_intermediaria = regiao_intermediaria)

print('-' * 100)
print(f'Municípios da mesorregião {seabra['mesorregiao-nome'].values[0]}:\n')
print(municipios_mesorregiao[['municipio-id', 'municipio-nome']])
print('-' * 100)
print(f'Municípios da região intermediária {seabra['regiao-intermediaria-nome'].values[0]}:\n')
print(municipios_regiao_intermediaria[['municipio-id', 'municipio-nome']])

https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/2906/municipios?view=nivelado&
https://servicodados.ibge.gov.br/api/v1/localidades/regioes-intermediarias/2910/municipios?view=nivelado&
----------------------------------------------------------------------------------------------------
Municípios da mesorregião Centro Sul Baiano:

     municipio-id               municipio-nome
0         2900108                       Abaíra
1         2900504                Érico Cardoso
2         2900603                      Aiquara
3         2901007                     Amargosa
4         2901205                        Anagé
5         2901304                      Andaraí
6         2901957                     Apuarema
7         2902005                      Aracatu
8         2902807              Barra da Estiva
9         2902906               Barra do Choça
10        2903508                   Belo Campo
11        2903706                     Boa Nova
12        2903953           Bom Jesus da

In [10]:
# agora, vamos checar quantas intersecções nós, o que pode ser feito em qualquer um dos dataframes
intersec1 = municipios_mesorregiao[
    municipios_mesorregiao['regiao-intermediaria-id'] == regiao_intermediaria]
intersec2 = municipios_regiao_intermediaria[
    municipios_regiao_intermediaria['mesorregiao-id'] == mesorregiao]

assert len(intersec1) == len(intersec2)
print(f'Existem {len(intersec1)} municípios que pertencem tanto à mesorregião {seabra['mesorregiao-nome'].values[0]} quanto à região intermediária {seabra['regiao-intermediaria-nome'].values[0]}.\n')
print(intersec1[['municipio-id', 'municipio-nome']])

Existem 14 municípios que pertencem tanto à mesorregião Centro Sul Baiano quanto à região intermediária Feira de Santana.

     municipio-id     municipio-nome
5         2901304            Andaraí
13        2904001            Boninal
41        2913002           Ibitiara
47        2915007             Itaeté
67        2919306            Lençóis
78        2920809  Marcionílio Souza
83        2921906             Mucugê
87        2922854      Nova Redenção
88        2923035     Novo Horizonte
91        2923506          Palmeiras
93        2924306              Piatã
108       2929909             Seabra
115       2932804             Utinga
117       2933406             Wagner


In [11]:
"""
Vamos montar agora um filtro de município para consultas agregadas
que traga apenas esses registros encontrados.
"""

# inicialmente, precisamos do id do nível territorial 'municipio'
nivel = ibge.identificador('nivel_geografico', 'municipio')

# agora vamos concatenar com os identificadores de município
# usando a sintaxe do próprio IBGE
filtro_municipios = ""
for mun in intersec1['municipio-id'].values:
    if filtro_municipios:
        filtro_municipios += ','
    else:
        filtro_municipios = nivel['id'] + '['
    filtro_municipios += str(mun)
filtro_municipios += ']'

# exibindo o resultado
print(filtro_municipios)

N6[2901304,2904001,2913002,2915007,2919306,2920809,2921906,2922854,2923035,2923506,2924306,2929909,2932804,2933406]


In [12]:
# resetando as configurações de exibição das tabelas
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.float_format')
pd.reset_option('display.max_colwidth')